# getting github api
https://docs.github.com/en/issues/planning-and-tracking-with-projects/automating-your-project/using-the-api-to-manage-projects?tool=curl

# setup

In [ ]:
import json
import requests
import logging as log
from pathlib import Path
from datetime import datetime, timedelta  #, timezone
from pandas import DataFrame, to_datetime  #, Timestamp, read_csv
log.basicConfig(level=log.DEBUG)

In [ ]:
# Set the URL and headers
url = "https://api.github.com/graphql"
TOKEN = Path('token.txt').read_text()
# token expires 2 jun 2024
headers = {'Authorization': f'Bearer {TOKEN}'}

In [ ]:
def printjson(x):
    print(json.dumps(x, indent=2))


def call_api(body, url=url, headers=headers):
    # Make the request
    response = requests.post(url, headers=headers, json={"query": body})
    # Check the response status code
    if response.status_code == 200:
        log.debug('call success 200')
        return response.json()
    else:
        log.warning(f'call unknown {response.status_code}')
        return response

# whos logged

In [ ]:
body = "query { viewer { login }}"
res = call_api(body)

In [ ]:
USR = res['data']['viewer']['login']
USR

# get project
## by number
Get project number by checking the link
`http://github.com/users/<project_owner_user>/projects/<proj_number>`

In [ ]:
body = 'query { user(login: "'+USR+'") {projectV2(number: 2) {id}}}'
res = call_api(body)

In [ ]:
pId = res['data']['user']['projectV2']['id']
pId

## first 20 projects

In [ ]:
body = 'query { user(login: "'+USR+'") {projectsV2(first: 20) {nodes {id title}}}}'
res = call_api(body)
printjson(res)

## All project fields

In [ ]:
body = 'query{ node(id: "'+pId+'''") { 
                ... on ProjectV2 { fields(first: 20) { nodes { 
                    ... on ProjectV2Field { id name } 
                    ... on ProjectV2IterationField { id name configuration { iterations { startDate id }}} 
                    ... on ProjectV2SingleSelectField { id name options { id name }}
                }}}}}'''

In [ ]:
res = call_api(body)
printjson(res)

In [ ]:
DataFrame(res["data"]["node"]["fields"]["nodes"])

In [ ]:
for statuses in filter(lambda x: x['name'] == 'Status', res["data"]["node"]["fields"]["nodes"]):
    print(statuses)
    display(DataFrame(statuses['options']))

In [ ]:
for sprints in filter(lambda x: x['name'] == 'Sprint', res["data"]["node"]["fields"]["nodes"]):
    print(sprints)
    display(DataFrame(sprints['configuration']['iterations']))

## Common project fields

In [ ]:
body = 'query{ node(id: "'+pId+'''") { 
            ... on ProjectV2 { fields(first: 20) { nodes { 
                ... on ProjectV2FieldCommon { id name }}}}}}'''

In [ ]:
res = call_api(body)

In [ ]:
DataFrame(res['data']['node']['fields']['nodes'])

## [Finding information about items in a project](https://docs.github.com/en/issues/planning-and-tracking-with-projects/automating-your-project/using-the-api-to-manage-projects?tool=curl#finding-information-about-items-in-a-project)

In [ ]:
body = 'query{ node(id: "'+pId+'''") { 
            ... on ProjectV2 { items(first: 20) { nodes{ id fieldValues(first: 8) { nodes{ 
                ... on ProjectV2ItemFieldTextValue { text field { ... on ProjectV2FieldCommon {  name }}} 
                ... on ProjectV2ItemFieldDateValue { date field { ... on ProjectV2FieldCommon { name }}} 
                ... on ProjectV2ItemFieldSingleSelectValue { name field { 
                ... on ProjectV2FieldCommon { name }}}}}
                content{ 
                    ... on DraftIssue { title body } 
                    ... on Issue { title assignees(first: 10) { nodes{ login }}} 
                    ... on PullRequest { title assignees(first: 10) { nodes{ login }}}}
            }}}}}'''

In [ ]:
res = call_api(body)
#res

### Parse logic
    'id'
    'fieldValues'
        'name'  'field': {'name': 'Status'}
        'date'  'field': {'name': 'Start'}
                'field': {'name': 'Due'}
    'content': 'title':
               'assignees': {'nodes': [{'login':

In [ ]:
for node in res['data']['node']['items']['nodes']:
    print()
    #print(node)
    #print(node.keys())
    print('id', node['id'])
    # content
    if 'content' in node.keys():
        if 'title' in node['content'].keys():
            print('title', node['content']['title'])
        if 'assignees' in node['content'].keys():
            for i, ass in enumerate(node['content']['assignees']['nodes']):
                print('assignee', i, ass['login'])
        if 'body' in node['content'].keys():
            print('body', node['content']['body'])
    # fieldValues
    if 'fieldValues' in node.keys():
        for f in node['fieldValues']['nodes']:
            if f:
                if 'date' in f.keys():
                    print(f['field']['name'], to_datetime(f['date']))
                elif 'name' in f.keys():
                    print(f['field']['name'], f['name'])
    break

# To ics calendar

    PRODID - A unique identifier for a calendar. :-//John Doe//My Calendar//EN
    VERSION - The version of the iCalendar standard that is used in the file.
    METHOD - The method used to create the iCalendar file.
    BEGIN:VCALENDAR - The start of the iCalendar file.
    END:VCALENDAR - The end of the iCalendar file.

    dtstart (ISO date)
    summary

Optional:

    location
    url
    dtend (ISO date), duration (ISO date duration)
    rdate, rrule
    category, description
    uid
    geo (latitude, longitude)
    attendee (partstat, role), contact, organizer
    attach
    status

In [ ]:
with open('project2calendar.ics', 'wt') as afile:
    afile.writelines([
        'BEGIN:VCALENDAR\n',
        'VERSION:2.0\n',
        'PRODID:-//fdobad//ical v1.0//EN\n'
    ])
    for node in res['data']['node']['items']['nodes']:
        afile.write('BEGIN:VEVENT\n')
        # content
        if 'content' in node.keys():
            if 'title' in node['content'].keys():
                title = node['content']['title'] if node['content']['title'] else 'no title'
                afile.writelines(f'SUMMARY:"{title.encode()}"\n')    
            if 'assignees' in node['content'].keys():
                for i, ass in enumerate(node['content']['assignees']['nodes']):
                    usr = ass['login']
                    afile.writelines(f'ATTENDEE:"{usr.encode()}"\n')
            #if 'body' in node['content'].keys():
            #    descr = str(node['content']['body']).encode( errors="replace")
            #    afile.write(f'DESCRIPTION: "{descr}"\n')
        # fieldValues
        if 'fieldValues' in node.keys():
            for f in node['fieldValues']['nodes']:
                if f:
                    if 'date' in f.keys():
                        if f['field']['name'] == 'Start':
                            dts = to_datetime(f['date']) if to_datetime(f['date']) else datetime.now()
                            dts = dts.strftime('%Y%m%dT%H%M%SZ')
                            afile.writelines(f'DTSTART:{dts}\n')
                        elif f['field']['name'] == 'Due':
                            dte = to_datetime(f['date']) if to_datetime(f['date']) else datetime.now()+timedelta(days=7)
                            dte = dte.strftime('%Y%m%dT%H%M%SZ')
                            afile.writelines(f'DTEND:{dte}\n')
                    elif 'name' in f.keys():
                        st = f['name'] if f['name'] else 'no status'
                        afile.writelines(f'STATUS:{st.encode()}\n')
        afile.write(f'UID:{node["id"]}\n')
        afile.write('END:VEVENT\n')
    afile.write('END:VCALENDAR\n')